# Import

In [1]:
import os
import torch
import pytorch_lightning as pl
from time import time_ns
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from rnaquanet.network.graph_regression_network import GraphRegressionNetwork
from rnaquanet.network.grn_data_module import GRNDataModule
from rnaquanet.utils.rnaquanet_config import RnaquanetConfig
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from torch.optim import Adam
from torch.nn import (
    BatchNorm1d,
    Identity,
    ReLU,
    LeakyReLU,
    Linear,
    MSELoss
)
import torch.nn.functional as F
from torch_geometric.nn import (
    GATConv,
    GCNConv,
    Sequential,
    global_mean_pool,
    BatchNorm,
    TransformerConv
)
from torch_geometric.nn.models import (
    GAT
)
from torch_geometric.loader import DataLoader
import numpy as np
from rnaquanet.data.preprocessing.hdf5_utils import load_data_from_hdf5
from IPython.display import clear_output
from tqdm import tqdm
import matplotlib.pyplot as plt
from rnaquanet.network.h5_graph_dataset import H5GraphDataset
import math

In [2]:
seg1_train_data = H5GraphDataset('/app/data/segments_1_coords/train.h5').__enter__()

seg1_val_data = H5GraphDataset('/app/data/segments_1_coords/val.h5').__enter__()
seg1_test_data = H5GraphDataset('/app/data/segments_1_coords/test.h5').__enter__()

seg2_val_data = H5GraphDataset('/app/data/segments_2_coords/val.h5').__enter__()
seg2_test_data = H5GraphDataset('/app/data/segments_2_coords/test.h5').__enter__()

seg3_val_data = H5GraphDataset('/app/data/segments_3_coords/val.h5').__enter__()
seg3_test_data = H5GraphDataset('/app/data/segments_3_coords/test.h5').__enter__()

ares_val_data = H5GraphDataset('/app/data/ares/val.h5').__enter__()
ares_test_data = H5GraphDataset('/app/data/ares/test.h5').__enter__()

dataset = 'seg1'

In [3]:
def plot_loss_chart(losses, epoch):
    bars = len(losses.values()) # num of bars
    epochs = range(1, epoch+2)
    width = 1.0/bars * 0.75
    multiplier = 0
    _, ax = plt.subplots(layout='constrained')

    for key, value in losses.items():
        x = np.arange(len(value['losses']))

        loss_values = value['losses']
        offset = width * multiplier
        rects = ax.bar(x + offset, loss_values, width, label=key, color=value['color'], alpha=0.5)
        ax.bar_label(rects, fmt='%0.0f', padding=3)
        ax.plot(x+((bars-1)/2)*width, loss_values, color=value['color'], alpha=0.5)
        multiplier += 1

    x = np.arange(len(epochs))
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.set_xticks(x + ((bars-1)/2)*width, epochs)
    ax.legend()
    plt.savefig(os.path.join('train', f'{dataset}.png'))
    plt.show()


def get_desc(epoch, losses):
    desc = f'Epoch {epoch+1}\n'
    for key, value in losses.items():
        if value['losses']:
            desc += f'Prev {key} {value["losses"][-1]:.2f}\tBest {key} {np.min(value["losses"]):.2f}\n'
    return desc.rstrip('\n')

def process_epoch(
        epoch, 
        batch_size, 
        device, 
        model,
        loss_fn,
        mse_no_reduce,
        optimizer,
        patience, 
        
        losses, key,
        best_model, best_loss,

        save_values = False,
        plot = True
    ):
    step = []
    values = {'predicted': [], 'true': []}
    assert(not losses[key]['train'] or not save_values)

    if losses[key]['data'].max_iterations != None:
        losses[key]['data'].shuffle()
    

    with tqdm(total=len(losses[key]['data']), desc=f'{key.capitalize()} epoch {epoch+1}', position=0, leave=True) as progressbar:
        if not losses[key]['train']:
            with torch.no_grad():
                for item in DataLoader(losses[key]['data'], batch_size=batch_size, shuffle=False, num_workers=4):
                    item = item.to(device)
                    y_pred = model(x=item.x, edge_index=item.edge_index, edge_attr=item.edge_attr, batch=item.batch).view(-1)
                    step.extend(mse_no_reduce(y_pred, item.y).cpu().tolist())
                    if save_values:
                        values['predicted'].extend(y_pred.cpu().tolist())
                        values['true'].extend(item.y.view(-1).cpu().tolist())
                    progressbar.update(item.y.cpu().shape[0])
        else:
            for item in DataLoader(losses[key]['data'], batch_size=batch_size, shuffle=(losses[key]['data'].max_iterations == None), num_workers=4):
                item = item.to(device)
                y_pred = model(x=item.x, edge_index=item.edge_index, edge_attr=item.edge_attr, batch=item.batch).view(-1)
                loss = loss_fn(y_pred, item.y)
                step.extend(mse_no_reduce(y_pred, item.y).cpu().tolist())
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                progressbar.update(item.y.cpu().shape[0])
        
    losses[key]['losses'].append(np.mean(step))
    if save_values:
        losses[key]['values'].append(values)
    clear_output(wait=True)
    print(get_desc(epoch, losses))
    if plot:
        plot_loss_chart(losses, epoch)

    index = np.argmin(losses[key]['losses'])
    if losses[key]['patience']:
        if losses[key]['losses'][-1] < best_loss:
            return model.state_dict().copy(), losses[key]['losses'][index], False
    return best_model, best_loss, losses[key]['patience'] and index <= (epoch-patience)

In [10]:
torch.set_float32_matmul_precision('high')
torch.manual_seed(2137)
device = torch.device('cuda:0')

max_epochs = 1000
batch_size = 32
patience = 3

model = Sequential('x, edge_index, edge_attr, batch', [
    (GATConv(in_channels=99, out_channels=256, heads=4, dropout=0.5), 'x, edge_index, edge_attr -> x'),
    (BatchNorm(in_channels=256*4), 'x -> x'),
    (ReLU(), 'x -> x'),
    (GATConv(in_channels=256*4, out_channels=256, heads=8, dropout=0.5), 'x, edge_index, edge_attr -> x'),
    (BatchNorm(in_channels=256*8), 'x -> x'),
    (ReLU(), 'x -> x'),

    (GCNConv(in_channels=256*8, out_channels=256), 'x, edge_index -> x'),
    (global_mean_pool, 'x, batch -> x'),

    (Linear(in_features=256, out_features=64), 'x -> x'),
    (ReLU(), 'x -> x'),
    (Linear(in_features=64, out_features=1), 'x -> x'),
]).to(device)

loss_fn = MSELoss(reduction='mean').to(device)
mse_no_reduce = MSELoss(reduction='none').to(device)
optimizer = Adam(model.parameters(), lr=1e-3)

losses = {
    'seg1 train': {
        'data': seg1_train_data,
        'losses': [],
        'color': 'red',
        'train': True,
        'patience': False
    },
    'seg1 val': {
        'data': seg1_val_data,
        'losses': [],
        'color': 'blue',
        'train': False,
        'patience': True,
        'values': []
    },
}

best_model = None
best_loss = float('inf')

for epoch in range(max_epochs):
    for key in losses.keys():
        best_model, best_loss, cond = process_epoch(epoch, batch_size, device, model, loss_fn, mse_no_reduce, optimizer, patience, losses, key, best_model, best_loss)
        if cond:
            break
    else:
        continue
    break

torch.save(best_model, os.path.join('model', f'{dataset}.pt'))

In [11]:
model.load_state_dict(best_model)

<All keys matched successfully>

In [8]:

with torch.no_grad():
    n = 0
    print(f'predicted\tground truth\tdiff')
    for item in DataLoader(losses['seg1 val']['data'], batch_size=1, shuffle=True, num_workers=4):
        item = item.to(device)
        y_pred = model(x=item.x, edge_index=item.edge_index, edge_attr=item.edge_attr, batch=item.batch).view(-1)
        print(f'{y_pred.cpu().item()}\t{item.y.cpu().item()}\t{y_pred.cpu().item()-item.y.cpu().item()}')
        n += 1
        if n == 10:
            break

predicted	ground truth	diff
6.58615779876709	0.8009999990463257	5.785157799720764
5.505395889282227	2.2249999046325684	3.280395984649658
5.700244903564453	1.9639999866485596	3.7362449169158936
5.0140886306762695	2.302999973297119	2.7110886573791504
8.450472831726074	2.2809998989105225	6.169472932815552
5.362732410430908	4.642000198364258	0.7207322120666504
5.826169967651367	9.298999786376953	-3.472829818725586
6.755646705627441	1.781999945640564	4.973646759986877
3.981149911880493	6.171999931335449	-2.190850019454956
4.908929347991943	4.124000072479248	0.7849292755126953


In [12]:
losses = {
    'ares val': {
        'data': ares_val_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'ares test': {
        'data': ares_test_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg1 val': {
        'data': seg1_val_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg1 test': {
        'data': seg1_test_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg2 val': {
        'data': seg2_val_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg2 test': {
        'data': seg2_test_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg3 val': {
        'data': seg3_val_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
    'seg3 test': {
        'data': seg3_test_data,
        'losses': [],
        'color': '',
        'train': False,
        'patience': False,
        'values': []
    },
}

for key in losses.keys():
    process_epoch(0, batch_size, device, model, loss_fn, mse_no_reduce, optimizer, patience, losses, key, best_model, best_loss, save_values=True, plot=False)
    values = losses[key]['values'].pop()
    plt.boxplot([values['predicted'], values['true']], labels=['Predicted', 'Ground Truth'])
    plt.title(f'{key.capitalize()} Values Box Plot (Trained on Seg1)')
    plt.ylabel('Values')
    plt.savefig(os.path.join('values', f'{dataset}_{key.replace(" ", "_")}.png'))
    plt.close()

plt.figure(figsize=(10,6))
plt.bar(list(losses.keys()), list(map(lambda l: l['losses'][0], losses.values())))
plt.title(f'Average Loss Across Datasets (Trained on Seg1)')
plt.ylabel('Loss')
plt.savefig(os.path.join('loss', f'{dataset}.png'))
plt.close()


Epoch 1
Prev ares val 11.87	Best ares val 11.87
Prev ares test 17.06	Best ares test 17.06
Prev seg1 val 12.70	Best seg1 val 12.70
Prev seg1 test 12.44	Best seg1 test 12.44
Prev seg2 val 33.95	Best seg2 val 33.95
Prev seg2 test 33.86	Best seg2 test 33.86
Prev seg3 val 67.82	Best seg3 val 67.82
Prev seg3 test 74.12	Best seg3 test 74.12


In [13]:
plt.figure(figsize=(10,6))
rects = plt.bar(list(losses.keys()), list(map(lambda l: l['losses'][0], losses.values())))
plt.bar_label(rects, fmt='%0.0f', padding=3)
plt.title(f'Average Loss Across Datasets (Trained on Seg1)')
plt.ylabel('Loss')
plt.savefig(os.path.join('loss', f'{dataset}.png'))
plt.close()